# Using Dask on Ray with the Ensemble

[Ray](https://docs.ray.io/en/latest/ray-overview/index.html) is an open-source unified framework for scaling AI and Python applications. Ray provides a scheduler for Dask ([dask_on_ray](https://docs.ray.io/en/latest/ray-more-libs/dask-on-ray.html)) which allows you to build data analyses using Dask’s collections and execute the underlying tasks on a Ray cluster. We have found with TAPE that the Ray scheduler is often more performant than Dasks scheduler. Ray can be used on TAPE using the setup shown in the following example.

In [ ]:
import ray
from ray.util.dask import enable_dask_on_ray, disable_dask_on_ray
from tape import Ensemble
from tape.analysis.structurefunction2 import calc_sf2

context = ray.init()

# Use the Dask config helper to set the scheduler to ray_dask_get globally,
# without having to specify it on each compute call.
enable_dask_on_ray()

We import ray, and just need to invoke two commands. `context = ray.init()` starts a local ray cluster, and we can use this context object to retrieve the url of the ray dashboard, as shown below. `enable_dask_on_ray()` is a dask configuration function that sets up all Dask work to use the established Ray cluster.

In [ ]:
print(context.dashboard_url)

For TAPE, the only needed change is to specify `client=False` when initializing an `Ensemble` object. Because the Dask configuration has been set, the Ensemble will automatically use the established Ray cluster.

In [ ]:
ens = Ensemble(client=False)  # Do not use a client

From here, we are free to work with TAPE as normal.

In [ ]:
ens.from_dataset("s82_qso")
ens.source = ens.source.repartition(npartitions=10)
ens.batch(
    calc_sf2, use_map=False
)  # use_map is false as we repartition naively, splitting per-object sources across partitions

## Timing Comparison

As mentioned above, we generally see that Ray is more performant than Dask. Below is a simple timing comparison.

### Ray Timing

In [ ]:
%%time

ens = Ensemble(client=False)  # Do not use a client
ens.from_dataset("s82_qso")
ens.source = ens.source.repartition(npartitions=10)
ens.batch(calc_sf2, use_map=False)

### Dask Timing

In [ ]:
disable_dask_on_ray()  # unsets the dask_on_ray configuration settings

In [ ]:
%%time

ens = Ensemble()
ens.from_dataset("s82_qso")
ens.source = ens.source.repartition(npartitions=10)
ens.batch(calc_sf2, use_map=False).compute()